# FYV Model Prototyping
Ravi Dayabhai

## Set up

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
RANDOM_STATE = 614

In [ ]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from modules import load, custom, pipeline

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, HuberRegressor
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import\
    StandardScaler, OneHotEncoder,RobustScaler, FunctionTransformer, PolynomialFeatures
from sklearn.metrics import explained_variance_score, max_error, r2_score
import statsmodels.api as sm

pd.set_option('display.max_columns', None)

In [ ]:
# Choose target variable
TARGET_VARIABLE = 'invoice_total_revenue'

# Model training switch
TRAINING_SWITCH = False

# Train-test split
TEST_PCT = .3

## Import Data

In [ ]:
# Load data
df = load.load_data('./queries/first_year_transactions.sql')
raw_data = df.copy()

In [ ]:
# Clean data
df = load.clean_data(df)
df.info()

In [ ]:
# Inspect cleaned data summary
meta_df = pd.concat([df.dtypes.rename('types'), 
                     df.isnull().mean().rename('missing_data'), 
                     df.nunique(dropna=False).rename('cardinality'),
                     df.var().rename('var')
                    ], 
                    axis=1).sort_values('missing_data', ascending=False)
meta_df

## EDA

**THIS SECTION IS INCOMPLETE.**

TODO: I plan on returning to do EDA on my own once I've set up the downstream pipeline properly. In the interim, I'm leaning on Danny's insights from his EDA.

In [ ]:
from pandas_profiling import ProfileReport

pr = ProfileReport(df)
pr.to_file(output_file="./data/df_profile.html")
pr

### Target Variable

On the face of it, costs and revenues tend to track one another and net revenue is the difference of these (as we should expect):

In [ ]:
income_statement = df[['segment',
                       'bill_total_sans_passthrough', 
                       'invoice_total_revenue', 
                       'actual_net_revenue']]
(income_statement['actual_net_revenue'] - (income_statement['invoice_total_revenue'] - income_statement['bill_total_sans_passthrough'])).describe()

In [ ]:
income_statement.corr()['actual_net_revenue']

As Danny mentioned, this is likely due to the bulk discounting (lower take rate) for our larger clients.

In [ ]:
# Isolate feature columns
feature_cols = df.columns.difference(['bill_total_sans_passthrough', 
                                      'invoice_total_revenue', 
                                      'actual_net_revenue'])
feature_df = df[feature_cols]
feature_df.head()

## Model Pipeline

In [ ]:
# Split features from target
X, y = pipeline.data_prep(df, target_feature=TARGET_VARIABLE)

In [ ]:
df.columns

In [ ]:
X.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=TEST_PCT, 
                                                    stratify=X['segment'], 
                                                    random_state=RANDOM_STATE)
print(f"Training data (train + CV) dimensions: {X_train.shape}")
print(f"Testing data (true hold-out) dimensions: {X_test.shape}")

### Data Transformation

In [ ]:
# Split features by transforms
FUNC_TRANSFORM_FEATURES = ['raw_piers_estimated_commodity_value']
PREENCODED_FEATURES = ['import_or_exporter_Exporter', 'import_or_exporter_Importer']

# Function transform
func_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('func', FunctionTransformer(validate=True))
])

# Define transforms on numeric, non-function transformed, non-indicators
numeric_features = X.select_dtypes(np.number).columns[~X.select_dtypes(np.number).columns.isin(PREENCODED_FEATURES + FUNC_TRANSFORM_FEATURES)]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler()),
    ('poly', PolynomialFeatures(interaction_only=True, include_bias=False))
])

# Define transforms on categorical types
categorical_features = X.select_dtypes(['object', 'category', 'bool']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('zero_var', pipeline.ZeroVariance(near_zero=True)),
    ('correlation', pipeline.FindCorrelation(threshold=0.9)),
])

# Construct ColumnTransformer object
preprocessor = ColumnTransformer(
    transformers=[
        ('func_transf', func_transformer, FUNC_TRANSFORM_FEATURES),
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough',
    verbose=False
)

### Preview Transformation

The following dataframe is the output of running the preprocessing column transformation on `X_test`. **Note**: This is *not* necessarily the dataframe that is passed to the predictor since we are doing a GridSearch over the *entire* pipeline process (i.e., transformation + model selection/tuning).

In [ ]:
# Show transformed data
pp_array = preprocessor.fit_transform(X_train, y_train)
transformed_df = pd.DataFrame(pp_array)

# Recover column labels from transformers
enc_categorical_features = preprocessor.transformers_[2][1]['correlation'].get_feature_names(
    preprocessor.transformers_[2][1]['zero_var'].get_feature_names(
        preprocessor.transformers_[2][1]['onehot'].get_feature_names(categorical_features)
    )
)

enc_numeric_features = preprocessor.transformers_[1][1]['poly'].get_feature_names(numeric_features)

transformed_df.columns = FUNC_TRANSFORM_FEATURES + list(enc_numeric_features) + list(enc_categorical_features) + PREENCODED_FEATURES

# Preview transformed data
print(transformed_df.shape)
transformed_df.head(10)

In [ ]:
# Check that features aren't duped in transformation
assert len(set(transformed_df.columns)) == transformed_df.shape[1]

### Training & Hyperparameter Tuning

In [ ]:
# Model training pipeline
if TRAINING_SWITCH:

    # Gridsearch hyperparameter values
    param_grid = [
  
        # Linear models
        {'predictor__regressor': [LinearRegression()],
         'preprocess__num__poly__degree': [1,2],
         'preprocess__func_transf__func__func': [None, np.sqrt]
        },
        
        {'predictor__regressor': [LinearRegression()],
         'preprocess__num__poly__degree': [1,2],
         'preprocess__func_transf__func__func': [None, np.sqrt],
         'predictor__func': [np.log],
         'predictor__inverse_func': [np.exp],
        },
        
        {'predictor__regressor': [Ridge()],
         'preprocess__num__poly__degree': [1,2],
         'preprocess__func_transf__func__func': [None, np.sqrt],
         'predictor__regressor__alpha': [0.1, 0.5, 1, 2, 5, 10]
        },
        
        {'predictor__regressor': [Ridge()],
         'preprocess__num__poly__degree': [1,2],
         'preprocess__func_transf__func__func': [None, np.sqrt],
         'predictor__regressor__alpha': [0.1, 0.5, 1, 2, 5, 10],
         'predictor__func': [np.log],
         'predictor__inverse_func': [np.exp]
        },

        # Non-linear models
        {'predictor__regressor': [AdaBoostRegressor()],
         'predictor__regressor__learning_rate': [.01, .1, 1],
         'preprocess__num__scaler': [None]
        },

        {'predictor__regressor': [RandomForestRegressor()],
         'predictor__regressor__min_samples_split': [.05, .10, .25],
         'preprocess__num__scaler': [None]
        },
        
        # Compare results to naive model
        {'predictor': [DummyRegressor(strategy='mean')]
        }

    ]

    
    # Feature selection and model selection pipeline
    model_build_pipe = Pipeline([
        
        # Do preprocessing
        ('preprocess', preprocessor),
        
        # Reduce dimensions
        ('dim_reduce', 'passthrough'),
        
        # Tune hyperparameters
        ('predictor', TransformedTargetRegressor())
        
    ], verbose=False)

    # Do hyperparamter grid search over entire pipeline
    model = GridSearchCV(model_build_pipe, 
                         param_grid=param_grid, 
                         scoring='r2', 
                         cv=5,
                         n_jobs=-1,
                         verbose=3,
                         return_train_score=True)

    ## Train model & get best parameters and scores
    model.fit(X_train, y_train)

else:
    model = load.model_loader('models/invoice_total_revenue_model (2020-07-20 20:49:00.268103).skmodel')

# See model results
model_results = pd.DataFrame(model.cv_results_)

In [ ]:
# View results
ignore_cols = custom.substring_list_match(model_results.columns, ['time'])
model_results.drop(columns=ignore_cols).sort_values(['rank_test_score']).head()

In [ ]:
# Make sure all models returned results
display(model_results[model_results['mean_test_score'].isnull()])
assert model_results['mean_test_score'].notnull().all(), "Some models failed to run: check model and transformation specs!"

## Model Evaluation

In [ ]:
# Test best fit model predictions
y_pred = model.predict(X_test)

# View best model
model.best_estimator_

### Evaluation Metrics

In [ ]:
# Various evaluation metrics on model
r2, max_err = r2_score(y_test, y_pred), max_error(y_test, y_pred)

print(f"R^2: {r2}")
print(f"Max Error: {max_err}")

In [ ]:
predicted = cross_val_predict(model.best_estimator_, X_test, y_test, cv=3)

fig, ax = plt.subplots()
ax.scatter(y_test, predicted, alpha=0.3)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4, alpha=0.8)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

## Model Archiving

In [ ]:
# Switch to save model
SAVER_SWITCH = False
if SAVER_SWITCH:
    prefix = y.name + '_model '
    load.model_saver(model, custom_prefix=prefix)
    model_results.to_csv(f"./models/cv_results/{y.name}_model ({datetime.datetime.now()}).csv")